# <p style="text-align: center;"> Variational Linear Systems Code </p> 
<p style="text-align: center;"> Ryan LaRose </p>

This notebook briefly demonstrates the current state of the Variational Linear Systems (VLS) code. All code is contained in `vls_pauli.py`, which defines a `PauliSystem` class.

In [82]:
# imports
import time

import numpy as np

from vls_pauli import PauliSystem

# Creating a Linear System of Equations

A `PauliSystem` consists of a matrix of the form
\begin{equation}
A = \sum_{k = 1}^{K} c_k \sigma_k
\end{equation}
where $c_k$ are complex coefficients and $\sigma_k$ are strings of Pauli operators. In code, we represent the matrix $A$ as arrays of strings corresponding to Pauli operators. For example, to represent the Pauli operators
\begin{align}
\sigma_1 &= \sigma_X \otimes \sigma_Y \otimes \sigma_X \otimes \sigma_Z \\
\sigma_2 &= \sigma_Z \otimes \sigma_X \otimes \sigma_I \otimes \sigma_Y
\end{align}
we would write:

In [83]:
# specify the pauli operators of the matrix
Amat_ops = np.array([["X", "Y", "X", "Z"],
                     ["Z", "X", "I", "Y"]])

Coefficients $c_k$ are stored similarly as arrays of complex values:

In [84]:
# specify the coefficients of each term in the matrix
Amat_coeffs = np.array([0.5 + 0.5j, 1 - 2j])

Finally, the solution vector
\begin{equation}
|b\rangle = U |0\>
\end{equation}
is represented by the unitary $U$ that (efficiently) prepares $|b\rangle$ from the ground state. For example, the unitary $U$ could be
\begin{equation}
U = \sigma_x \otimes \sigma_x \otimes \sigma_I \otimes \sigma_X,
\end{equation}
which we would represent in code as:

In [85]:
# specify the unitary that prepares the solution vector b
Umat_ops = np.array(["X", "X", "I", "X"])

To create `PauliSystem`, we can then simply feed in `Amat_coeffs`, `Amat_ops`, and `Umat_ops`.

In [86]:
# create a linear system of equations
system = PauliSystem(Amat_coeffs, Amat_ops, Umat_ops)

# Working with a `PauliSystem`

The `PauliSystem` class can tell basic information about the system:

In [87]:
print("Number of qubits in system:", system.num_qubits())
print("Size of matrix:", system.size())

Number of qubits in system: 4
Size of matrix: (16, 16)


To see the actual matrix representation of the system (in the computational basis), we can do:

In [88]:
# get the matrix of the system
matrix = system.matrix()
print(matrix)

[[ 0. +0.j   0. +0.j   0. +0.j   0. +0.j   0. +0.j  -2. -1.j   0. +0.j
   0. +0.j   0. +0.j   0. +0.j   0. +0.j   0. +0.j   0. +0.j   0. +0.j
   0.5-0.5j  0. +0.j ]
 [ 0. +0.j   0. +0.j   0. +0.j   0. +0.j   2. +1.j   0. +0.j   0. +0.j
   0. +0.j   0. +0.j   0. +0.j   0. +0.j   0. +0.j   0. +0.j   0. +0.j
   0. +0.j  -0.5+0.5j]
 [ 0. +0.j   0. +0.j   0. +0.j   0. +0.j   0. +0.j   0. +0.j   0. +0.j
  -2. -1.j   0. +0.j   0. +0.j   0. +0.j   0. +0.j   0.5-0.5j  0. +0.j
   0. +0.j   0. +0.j ]
 [ 0. +0.j   0. +0.j   0. +0.j   0. +0.j   0. +0.j   0. +0.j   2. +1.j
   0. +0.j   0. +0.j   0. +0.j   0. +0.j   0. +0.j   0. +0.j  -0.5+0.5j
   0. +0.j   0. +0.j ]
 [ 0. +0.j  -2. -1.j   0. +0.j   0. +0.j   0. +0.j   0. +0.j   0. +0.j
   0. +0.j   0. +0.j   0. +0.j  -0.5+0.5j  0. +0.j   0. +0.j   0. +0.j
   0. +0.j   0. +0.j ]
 [ 2. +1.j   0. +0.j   0. +0.j   0. +0.j   0. +0.j   0. +0.j   0. +0.j
   0. +0.j   0. +0.j   0. +0.j   0. +0.j   0.5-0.5j  0. +0.j   0. +0.j
   0. +0.j   0. +0.j ]
 [ 0. +0.

We can also see the solution vector $|b\rangle$ by doing:

In [89]:
b = system.vector()
print(b)

[0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j
 0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j]


# Creating an Ansatz

Initially, the `PauliSystem` ansatz for $V$ is an empty circuit:

In [90]:
system.ansatz

We are free to pick whatever ansatz we wish. The method `PauliSystem.make_ansatz_circuit` is currently set to make an alternating two-qubit gate ansatz.

In [91]:
system.make_ansatz_circuit()
system.ansatz # note: circuits aren't printed to allow for scrollable outputs

1: ───X^Symbol("0")────Y^Symbol("1")────Z^Symbol("2")────@───X^Symbol("6")────Y^Symbol("7")────Z^Symbol("8")────@───X^Symbol("39")───Y^Symbol("40")───Z^Symbol("41")───────X───X^Symbol("45")───Y^Symbol("46")───Z^Symbol("47")───────X───
                                                         │                                                      │                                                          │                                                          │
2: ───X^Symbol("3")────Y^Symbol("4")────Z^Symbol("5")────X───X^Symbol("9")────Y^Symbol("10")───Z^Symbol("11")───X───X^Symbol("24")───Y^Symbol("25")───Z^Symbol("26")───@───┼───X^Symbol("30")───Y^Symbol("31")───Z^Symbol("32")───@───┼───
                                                                                                                                                                       │   │                                                      │   │
3: ───X^Symbol("12")───Y^Symbol("13")───Z^Symbol("14")───@───X^Symbol("18")───Y^Symbol("19")───Z^Symbol("20")───@───X^Symbol("27")───Y^Symbol("28")───Z^Symbol("29")───X───┼───X^Symbol("33")───Y^Symbol("34")───Z^Symbol("35")───X───┼───
                                                         │                                                      │                                                          │                                                          │
4: ───X^Symbol("15")───Y^Symbol("16")───Z^Symbol("17")───X───X^Symbol("21")───Y^Symbol("22")───Z^Symbol("23")───X───X^Symbol("36")───Y^Symbol("37")───Z^Symbol("38")───────@───X^Symbol("42")───Y^Symbol("43")───Z^Symbol("44")───────@───

This circuit contains 48 parameters (4 qubits x 2 "gates" / qubit x 6 parameters / gate). (Note that printing the circuit gets cut off in the notebook, scroll side to side to see the entire circuit.) For our simple example, we will chop off some of the gates to make the optimization easier:

In [92]:
system.ansatz = system.ansatz[:-9]
system.ansatz

1: ───X^Symbol("0")────Y^Symbol("1")────Z^Symbol("2")────@───X^Symbol("6")────Y^Symbol("7")────Z^Symbol("8")────
                                                         │
2: ───X^Symbol("3")────Y^Symbol("4")────Z^Symbol("5")────X───X^Symbol("9")────Y^Symbol("10")───Z^Symbol("11")───

3: ───X^Symbol("12")───Y^Symbol("13")───Z^Symbol("14")───@───X^Symbol("18")───Y^Symbol("19")───Z^Symbol("20")───
                                                         │
4: ───X^Symbol("15")───Y^Symbol("16")───Z^Symbol("17")───X───X^Symbol("21")───Y^Symbol("22")───Z^Symbol("23")───

# Computing the Cost

The local cost function is computed via the Hadamard Test. The local cost function can be written
\begin{equation}
C_1 = 1 - \frac{1}{n} \sum_{k = 1}^{K} \sum_{l = 1}^{K} w_{k, l} c_k c_l^* \sum_{j = 1}^{n} \langle V_{k, l}^{(j)} \rangle
\end{equation}
where
\begin{equation}
\langle V_{k, l}^{(j)} \rangle := \langle0^{\otimes n}| V^\dagger A_k^\dagger U P_j U^\dagger A_l V |0^{\otimes n}\rangle
\end{equation}
Thus we have $n K^2$ different circuits to compute the cost. An example of one (the $k = 0$, $l = 1$, $j = 1$ term) is shown below:

In [93]:
system.make_hadamard_test_circuit(system.ops[0], system.ops[1], 0, "real")

0: ───H─────────────────────────────────────────────────────────────────────────────────────────────────────────────@───@───@───@───────@───────@───@───@───H───M('z')───
                                                                                                                    │   │   │   │       │       │   │   │
1: ───────X^Symbol("0")────Y^Symbol("1")────Z^Symbol("2")────@───X^Symbol("6")────Y^Symbol("7")────Z^Symbol("8")────X───┼───┼───┼───X───@───X───Z───┼───┼────────────────
                                                             │                                                          │   │   │                   │   │
2: ───────X^Symbol("3")────Y^Symbol("4")────Z^Symbol("5")────X───X^Symbol("9")────Y^Symbol("10")───Z^Symbol("11")───────Y───┼───┼───X───────X───────X───┼────────────────
                                                                                                                            │   │                       │
3: ───────X^Symbol("12")───Y^Symbol("13")───Z^Symbol("14")───@───X^Symbol("18")───Y^Symbol("19")───Z^Symbol("20")───────────X───┼───────────────────────┼────────────────
                                                             │                                                                  │                       │
4: ───────X^Symbol("15")───Y^Symbol("16")───Z^Symbol("17")───X───X^Symbol("21")───Y^Symbol("22")───Z^Symbol("23")───────────────Z───X───────X───────────Y────────────────

To compute the cost, we can call `PauliSystem.cost` or `PauliSystem.eff_cost` (the latter exploits symmetries to compute the cost more efficiently) and pass in a set of angles to the ansatz gates:

In [94]:
angles = np.zeros(24)
start = time.time()
cost = system.eff_cost(angles)
end = time.time() - start
print("Local cost C_1 =", cost)
print("Time to compute cost =", end, "seconds")

-2.63235
Local cost C_1 = -2.63235
Time to compute cost = 0.6501102447509766 seconds
